In [1]:
#divide and conquer paradigm
# - divide into smaller subproblems
# - conquer subproblems recursively
# - combine solutions of subporblems into one for the original problem


# Px = points sorted by x
# Py = points sorted by y
# Recursively split into Q and R
# Determine smallest distance for each Q and R
# Get delta between smallest distance in Q and R
# Create Sy as the location of the (split - delta) - (split + delta)
# p and q are at most 7 positions apart in Sy

In [2]:
import numpy as np
import math

In [21]:
x = np.random.rand(200).round(4)
y = np.random.rand(200).round(4)
pairs = list(zip(x, y))

In [22]:
def euclidean_distance(p1, p2):
    x1 = p1[0]; x2 = p2[0]; y1 = p1[1]; y2 = p2[1]
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def brute_force(pairs):
    shortest_dist = euclidean_distance(pairs[0], pairs[1])
    closest_pair = (pairs[0], pairs[1])
    for i in pairs:
        for j in pairs:
            if i != j:
                dist = euclidean_distance(i, j)
                if dist < shortest_dist:
                    shortest_dist = dist
                    closest_pair = (i, j)
    return shortest_dist, closest_pair

def merge_sort_pairs(array, position):
    if len(array) == 1:
        return array
    else:
        m = len(array) // 2
        L = array[:m]
        R = array[m:]

        L = merge_sort_pairs(L, position=position)
        R = merge_sort_pairs(R, position=position)
        
        i = 0
        j = 0
        merged = []
        
    while (i < len(L)) and (j < len(R)):
        if L[i][position] <= R[j][position]:
            merged.append(L[i])
            i += 1
        else:
            merged.append(R[j])
            j += 1
    
    merged += L[i:]
    merged += R[j:]
    
    return merged

def closest_pair_split(px, py):
    
    # --- base case ---
    if len(px) <= 3:
        return brute_force(px)
    
    # --- split ---
    mx = len(px) // 2
    qx = px[:mx]
    rx = px[mx:]
    median = px[mx]
    
    qy = []; ry = []
    for p in py:
        if p[0] < median[0]:
            qy.append(p)
        else:
            ry.append(p)
    
    # --- find shortest dist on each side ---
    q_shortest = closest_pair_split(qx, qy)
    r_shortest = closest_pair_split(rx, ry)
    if q_shortest < r_shortest:
        shortest_dist, closest_pair = q_shortest
    else:
        shortest_dist, closest_pair = r_shortest
    
    # --- test if shortest was between split ---
    x_bar = qx[-1][0] #last x on left
    sy = [] #points along split
    for p in py: #test each point sorted by y to see if it is less than delta away from split
        if x_bar - shortest_dist < p[0]:
            if p[0] < x_bar + shortest_dist:
                sy.append(p)

    for i in range(len(sy) - 1):
        for j in range(i+1, min(i+7, len(sy))):
            p1 = sy[i]
            p2 = sy[j]
            dist = euclidean_distance(p1, p2)
            if dist < shortest_dist:
                shortest_dist = dist
                closest_pair = (p1, p2)
    
    return shortest_dist, closest_pair

def initial_sort(pairs):
    px = merge_sort_pairs(pairs, 0)
    py = merge_sort_pairs(pairs, 1)
    return px, py
    

def clever_closest_pair(pairs):
    px, py = initial_sort(pairs)
    return closest_pair_split(px, py)
    

In [23]:
brute_force(pairs)

(0.007623647421018358, ((0.6012, 0.2028), (0.6006, 0.2104)))

In [24]:
clever_closest_pair(pairs)

(0.007623647421018358, ((0.6012, 0.2028), (0.6006, 0.2104)))

In [25]:
%%timeit
brute_force(pairs)

193 ms ± 4.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%%timeit
clever_closest_pair(pairs)

11.6 ms ± 136 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
